In [ ]:
#
#    Notebook de cours MAP412 - Chapitre 6 - M. Massot 2020-2021 - Ecole polytechnique
#    ----------   
#    Résolution de la solution stationnaire - Explosion thermique
#    
#    Auteurs : L. Séries et M. Massot - (C) 2021
#   

In [ ]:
import time
import numpy as np

from scipy import optimize
from scipy.sparse import diags
from scipy.sparse.linalg import spsolve, eigs, factorized

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Résolution de la solution stationnaire - Explosion thermique

## Formulation du problème

On souhaite résoudre le problème elliptique constitué par l'équation de Poisson soumise à des conditions aux limites de type de Dirichlet et à un terme source non-linéaire de type explosion thermique :

$$
\left\{
\begin{aligned}
-\mathrm{d}_x^2 \theta(x) & = \lambda_{\mathrm{FK}} \exp(\theta(x)) &&x\in \Omega = ]0;2[, \\
        \theta(x) & =  0 && x\in \{0,2\},
\end{aligned}
\right.
$$

où $\lambda_{\mathrm{FK}}$ est le paramètre de Frank-Kamenetskii. 

In [ ]:
class thermal_explosion_model:
    
    def __init__(self, lamb, xmin, xmax, nx):
        self.lamb = lamb
        self.xmin = xmin
        self.xmax = xmax
        self.nx = nx
        self.dx = (xmax-xmin)/(nx+1)

    def fcn(self, theta):
        lamb = self.lamb
        nx = self.nx
        dx = self.dx
        oneoverdxdx = 1/dx**2
            
        lap = np.zeros(nx)
        lap[0]    = oneoverdxdx * (2*theta[0] - theta[1])
        lap[1:-1] = oneoverdxdx * (-theta[:-2] + 2*theta[1:-1] - theta[2:])
        lap[-1]   = oneoverdxdx * (-theta[-2] + 2*theta[-1])

        return lap - lamb*np.exp(theta)
    
    def jac(self, theta):
        lamb = self.lamb
        nx = self.nx
        dx = self.dx
        diagonals = [np.repeat(2/dx**2, nx) - lamb*np.exp(theta), np.repeat(-1/dx**2, nx-1), np.repeat(-1/dx**2, nx-1)]
        return diags(diagonals, [0, -1, 1])

In [ ]:
def newton(f, jac, x0, tol=1.e-10, max_iter=50):
    
    xk = np.copy(x0)
    
    res = np.zeros(max_iter+1)
    res[0] = np.linalg.norm(f(xk))
    
    incre = np.zeros(max_iter+1)
    incre[0] = 0
    
    # iteration de Newton        
    for k in range(max_iter):
        increment = spsolve(jac(xk).tocsr(), -f(xk)) 
        xk = xk + increment
        incre[k+1] = np.linalg.norm(increment)/np.linalg.norm(xk)
        reskp1 = np.linalg.norm(f(xk))
        print(f"Iteration nb {k+1:3d}: ||f(xk)|| = {reskp1}")
        res[k+1] = reskp1
        if ( np.linalg.norm(f(xk)) < tol ): break
 
    return xk, res[:k+2], incre[:k+2]

def modified_newton(f, jac, x0, tol=1.e-10, max_iter=50, freq=2):
    
    xk = np.copy(x0)
    
    res = np.zeros(max_iter+1)
    res[0] = np.linalg.norm(f(xk))
    
    incre = np.zeros(max_iter+1)
    incre[0] = 0
    
    # iteration de Newton        
    for k in range(max_iter):
        if k%freq == 0:
            jac_mat = jac(xk)  
            solve = factorized(jac_mat.tocsc())
        increment = solve(-f(xk)) 
        xk = xk + increment
        incre[k+1] = np.linalg.norm(increment)/np.linalg.norm(xk)
        reskp1 = np.linalg.norm(f(xk))
        print(f"Iteration nb {k+1:3d}: ||f(xk)|| = {reskp1}")
        res[k+1] = reskp1
        if ( np.linalg.norm(f(xk)) < tol ): break
    
    return xk, res[:k+2], incre[:k+2]

## Méthode de Newton

In [ ]:
# Valeur limite de lambda = 0.88 ! Bonnes valeurs : 0.1 - 0.5 - 0.85 - 0.878
lamb = 0.1
xmin = 0.
xmax = 2.
# nb of points including boundary conditions
nxib = 1002
nx = nxib-2

tem = thermal_explosion_model(lamb=lamb, xmin=xmin, xmax=xmax, nx=nx)
fcn = tem.fcn
jac = tem.jac

theta_ini = np.zeros(nx)

print(f"Algorithme de Newton")
max_iter=15
theta_sol, res, increment = newton(fcn, jac, theta_ini, tol=1.e-13, max_iter=max_iter)

jac_eq = jac(theta_sol)

eig_val_min = np.real(eigs(jac_eq, k=1, which='SR')[0])[0]
eig_val_max = np.real(eigs(jac_eq, k=1, which='LR')[0])[0]
print(f"\nConditionnement de la matrice Jacobienne à l'équilibre : {eig_val_max/eig_val_min}")
print(f"\nConditionnement de la matrice du Laplacien : {(2*(nx+1)/np.pi)**2}")

dx = (xmax-xmin)/(nxib-1)
x = np.linspace(0+dx, 1-dx, nx)

fig = make_subplots(rows=3, cols=1, vertical_spacing=0.12, 
                    subplot_titles=("Solution", "Evolution du résidu", "Evolution de l'incrément"))
fig.add_trace(go.Scatter(x=x, y=theta_sol, showlegend=False, 
                         line_color='cornflowerblue'), row=1, col=1)
fig.add_trace(go.Scatter(x=np.arange(res.size), y=res, showlegend=False, mode='lines+markers',
                         line=dict(color='cornflowerblue', dash='dot')), row=2, col=1)
fig.add_trace(go.Scatter(x=np.arange(1,increment.size), y=increment[1:], showlegend=False, mode='lines+markers',
                         line=dict(color='cornflowerblue', dash='dot')), row=3, col=1)

fig.update_xaxes(range=[-0.5,increment.size-0.5], row=2)    
fig.update_yaxes(type="log", exponentformat='e', row=2)   
fig.update_xaxes(range=[-0.5,increment.size-0.5], row=3)    
fig.update_yaxes(type="log", exponentformat='e', row=3)    
fig.update_layout(height=1200)
fig.show()

## Méthode de Newton modifiée avec ré-évaluation périodique du Jacobien 

Le but de cette seconde expérience est de résoudre le problème précédent mais en limitant le coût calcul de l'évaluation du Jacobien et de sa décomposition en ne le faisant que périodiquement voire une fois au début du calcul (paramètre "freq" pour fréquence de ré-évaluation de la matrice Jacobienne que l'on prends à max_iter si l'on ne veut qu'une évaluation au début du calcul).

Il faut effectuer le calcul avec $\lambda_{\mathrm{FK}}=0.1$ dans un premier temps où l'on constate qu'une évaluation au début du calcul est suffisante pour converger car le problème est relativement faiblement non linéaire et raisonnablement mal conditionné. Par contre, cela devient impossible pour $\lambda_{\mathrm{FK}}=0.878$, proche du point limite, du fait de la forte non-linéarité du système et de la forte augmentation du conditionnement.


In [ ]:
# Valeur limite de lambda = 0.88 ! Bonnes valeurs : 0.1 - 0.5 - 0.85 - 0.878
lamb = 0.2
xmin = 0.
xmax = 2.
# nb of points including boundary conditions
nxib = 1002
nx = nxib-2

tem = thermal_explosion_model(lamb=lamb, xmin=xmin, xmax=xmax, nx=nx)
fcn = tem.fcn
jac = tem.jac

theta_ini = np.zeros(nx)

print(f"Algorithme de Newton")
max_iter = 20
theta_sol, res, increment = newton(fcn, jac, theta_ini, tol=1.e-14, max_iter=max_iter)

print(f"\nAlgorithme de Newton modifié")
# on fixe ici la fréquence d'évaluation du jacobien et de sa décomposition
# si freq = max_iter alors on ne fait qu'une évaluation de Jacobien au début
freq = 3
theta_sol_mod, res_mod, increment_mod = modified_newton(fcn, jac, theta_ini, tol=1.e-14, max_iter=max_iter, freq=freq)

jac_eq = jac(theta_sol)

eig_val_min = np.real(eigs(jac_eq, k=1, which='SR')[0])[0]
eig_val_max = np.real(eigs(jac_eq, k=1, which='LR')[0])[0]
print(f"\nConditionnement de la matrice Jacobienne à l'équilibre : {eig_val_max/eig_val_min}")
print(f"\nConditionnement de la matrice du Laplacien  : {(2*(nx+1)/np.pi)**2}")

fig = make_subplots(rows=2, cols=1, vertical_spacing=0.12, 
                    subplot_titles=("Evolution du résidu", "Evolution de l'incrément"))
fig.add_trace(go.Scatter(x=np.arange(res.size), y=res, mode='lines+markers', name='Newton',
                         line=dict(color='cornflowerblue', dash='dot'), legendgroup = '1'), row=1, col=1)
fig.add_trace(go.Scatter(x=np.arange(res_mod.size), y=res_mod, mode='lines+markers', name='Newton modifié',
                        line=dict(color='crimson', dash='dot'), legendgroup = '1'), row=1, col=1)
fig.add_trace(go.Scatter(x=np.arange(1,increment.size), y=increment[1:], mode='lines+markers', name='Newton',
                         line=dict(color='cornflowerblue', dash='dot'), legendgroup = '2'), row=2, col=1)
fig.add_trace(go.Scatter(x=np.arange(1,increment_mod.size), y=increment_mod[1:], mode='lines+markers', name='Newton modifié',
                         line=dict(color='crimson', dash='dot'), legendgroup = '2'), row=2, col=1)

fig.update_xaxes(range=[-0.5,increment.size-0.5], row=1)    
fig.update_yaxes(type="log", exponentformat='e', row=1)   
fig.update_xaxes(range=[-0.5,increment.size-0.5], row=2)    
fig.update_yaxes(type="log", exponentformat='e', row=2)    
fig.update_layout(height=1000, legend_tracegroupgap=430, legend=dict(x=0.77, bgcolor='rgba(0,0,0,0)'))
fig.show()